In [1]:
# TODO
# catologue risks
# refer to hse (health and safety executive)
# We have caught carp, barbel, roach, rudd, perch, skimmer, bream, gudgeon, tench,
#

In [2]:
from IPython.display import display, Markdown

In [3]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [4]:
from scout.risk.models import RiskAssessment
from scout.risk.prompts import assessment_prompt

from langchain.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=RiskAssessment)
format_instructions = output_parser.get_format_instructions()

format_instructions

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"Risk": {"description": "Describes a risk.", "properties": {"hazard": {"description": "Descriptions of the hazard. A hazard is something that may cause harm or damage.", "title": "Hazard", "type": "string"}, "risks": {"description": "Risks associated with the hazard. A risk is the harm that may occur from the hazard. Describe each risk in a sentence.", "items": {"type": "string"}, "title": "Risks", "type": "array"}, "affects": {"description": "Who is at risk.", "items": {"enum": ["Young people", "Adult volunteers"], "type": 

In [5]:
import json
from scout.risk.prompts import user_prompt, assessment_prompt
from scout.risk.models import ExampleRiskAssessment
import itertools

from pprint import pprint

with open("assessment_examples.json") as f:
    examples = json.load(f)

examples = [ExampleRiskAssessment(**example).to_messages(user_prompt) for example in examples]


examples = list(itertools.chain(*examples))
# examples = []
pprint(examples)

[HumanMessage(content='\nLocation: The scout hut is large brick building with a hard wooden floor.\n\nActivity: Indoor active games.\n\nIdentify 9 or more risk(s).\n', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={}, name='48d3b298-4f9f-48f3-b125-8ee670de9f40', tool_calls=[{'name': 'RiskAssessment', 'args': {'activity': 'Indoor games', 'location': 'Scout Hut', 'risks': [{'hazard': 'Walls and items attached to walls (eg radiators)', 'risks': ['Collisions with walls causing injuries to people.'], 'affects': ['Young people'], 'controls': ['Mark a line on the floor about two metres before the walls at each end to show the safe zone. Explain the safe zone to players (for example, no one should run in the safe zone).', 'Consider the height and position of permanent fixings to walls, can they be changed.', 'Add additional safe zone marking around objects permanently fixed to walls where necessary.']}, {'hazard': 'Tables and chairs

In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
llm = model.with_structured_output(RiskAssessment, include_raw=True)
app = assessment_prompt.partial(examples=examples, n_min=7) | llm

args = {
    "activity": (
        "Pumpkin carving with tee light candles. "
        "Pumpkin carving will be done on tables in groups of 2, 2 groups per table."
    ),
    # "activity": "Variety of indoor games such as dodgeball, crab football etc.",
    # "activity": "Using a knife to whittle wood.",
    # "activity": "Rock paper scissors.",
    # "activity": (
    #     "Visiting 2 local chip shops and sample the chips. Scouts will need to cross roads and not "
    #     "get lost. Scouts will travel in groups of 4, each with one adult leader. Scouts should plan."
    # ),
    # "activity": "Scouts making pancakes in the kitchen.",
    # "location": "The scout kitchen is a medium sized kitchen similar to a typical family kitchen",
    # "location": (
    #     "The Queens Hall is large brick building with a hard wooden floor and a raised stage at the front. "
    #     "It contains a small modern kitchen, a number of exposed pillars to support the roof and stacked chairs around the perimeter."
    # ),
    "location": (
        "The Queens Hall is large brick building with a large room with a hard wooden floor with stacked chairs around the perimeter. "
        "It also contains a small modern kitchen."
    ),
    # "location": "Local urban area at night.",
}
risks = app.invoke(args)

In [7]:
print(len(risks["parsed"].risks))
display(Markdown(risks["parsed"].to_markdown()))

8


## Pumpkin carving with tea light candles - The Queens Hall

### Hazard
Sharp carving tools
### Affects
Young people

Adult volunteers
### Risks
Injuries from cuts or punctures.
### Controls
Adults to supervise young people when using carving tools, ensuring they are used safely.

Demonstrate the correct way to use tools and how to pass them safely between participants.

Limit the number of sharp tools in use at one time to reduce risk.




### Hazard
Tea light candles
### Affects
Young people

Adult volunteers
### Risks
Burns or fire hazards.
### Controls
Ensure that candles are placed in stable holders and monitored at all times.

Establish a safe distance between candles and flammable materials (including clothing).

Brief participants on fire safety and the importance of not leaning over lit candles.




### Hazard
Tables and chairs around the perimeter
### Affects
Young people

Adult volunteers
### Risks
Tripping or falling over stacked chairs or tables.
### Controls
Ensure that chairs are stored safely and not in walkways or areas where participants will be moving.

Keep the area around the tables clear of personal items and equipment.




### Hazard
Floor surface
### Affects
Young people

Adult volunteers
### Risks
Slips, trips, and falls due to spills or debris.
### Controls
Check the floor is clean and dry before starting activities.

Have a procedure for cleaning up any spills immediately to prevent accidents.




### Hazard
Kitchen area
### Affects
Young people

Adult volunteers
### Risks
Injuries from sharp items or burns from hot appliances.
### Controls
Limit access to the kitchen to adults or designated young people under supervision.

Ensure that any hot items (if used) are placed away from the activity area and monitored.




### Hazard
Behaviour during activity
### Affects
Young people

Adult volunteers
### Risks
Disruptive behaviour leading to accidents or injuries.
### Controls
Establish clear behaviour expectations and a code of conduct before starting the activity.

Adults to monitor behaviour and intervene if necessary to maintain safety.




### Hazard
Individual needs
### Affects
Young people
### Risks
Exclusion or injury due to lack of support.
### Controls
Consider individual needs when planning the activity, ensuring all young people can participate safely.

Provide additional support or adaptations as necessary for those with specific needs.




### Hazard
Inadequate supervision
### Affects
Young people
### Risks
Injuries or accidents due to lack of adult oversight.
### Controls
Ensure a sufficient number of adults are present to supervise the activity effectively, following appropriate adult-to-young person ratios.


